Importação de Bibliotecas

In [23]:
# Importação de bibliotecas
import requests  # Para acessar a API
import pandas as pd  # Para manipular os dados
import sqlite3  # Para armazenar os dados em SQLite

Definição da URL da API e função para consultar dados

In [24]:
# Definir a URL da API
url_api = "https://brasilapi.com.br/api/banks/v1"

# Função para acessar a API e retornar os dados
def acessar_api(url):
    try:
        response = requests.get(url)  # Fazendo a requisição GET
        response.raise_for_status()  # Verificando se houve erro na requisição
        return response.json()  # Retorna os dados no formato JSON
    except requests.exceptions.RequestException as e:
        print(f"Erro ao acessar a API: {e}")
        return None

Obter dados da API

In [25]:
# Chamar a função para acessar a API
dados = acessar_api(url_api)

# Verificar se os dados foram obtidos com sucesso
if dados:
    print("Dados extraídos com sucesso!")
else:
    print("Falha ao obter dados da API.")

Dados extraídos com sucesso!


Criar o DataFrame e tratar os dados

In [27]:
# Se os dados foram obtidos, criamos o DataFrame
if dados:
    df_principal = pd.DataFrame(dados)
    
    # Remover duplicatas e substituir valores nulos
    df_principal = df_principal.drop_duplicates()
    df_principal = df_principal.fillna('Desconhecido')

    # Padronizar os nomes das colunas
    df_principal.columns = ['ISPB', 'Nome_Banco', 'Codigo_Banco', 'Nome_Completo']

    # Normalizar o texto nas colunas de nome
    df_principal['Nome_Banco'] = df_principal['Nome_Banco'].str.upper()
    df_principal['Nome_Completo'] = df_principal['Nome_Completo'].str.upper()

    print("Dados tratados com sucesso!")
    display(df_principal.head())  # Exibindo as primeiras linhas para conferirmos

Dados tratados com sucesso!


,ISPB,Nome_Banco,Codigo_Banco,Nome_Completo
0,00000000,BCO DO BRASIL S.A.,1.0,BANCO DO BRASIL S.A.
1,00000208,BRB - BCO DE BRASILIA S.A.,70.0,BRB - BANCO DE BRASILIA S.A.
2,00038121,SELIC,Desconhecido,BANCO CENTRAL DO BRASIL - SELIC
3,00038166,BACEN,Desconhecido,BANCO CENTRAL DO BRASIL
4,00122327,SANTINVEST S.A. - CFI,539.0,"SANTINVEST S.A. - CREDITO, FINANCIAMENTO E INV..."


Tabelas filtradas

Tabela 1: Todos os Dados (Dados Brutos Tratados)

In [29]:
# Tabela 1: Todos os dados tratados
df_tabela1 = df_principal.copy()
print("Tabela 1: Todos os Dados Tratados")
display(df_tabela1.head())


Tabela 1: Todos os Dados Tratados


,ISPB,Nome_Banco,Codigo_Banco,Nome_Completo
0,00000000,BCO DO BRASIL S.A.,1.0,BANCO DO BRASIL S.A.
1,00000208,BRB - BCO DE BRASILIA S.A.,70.0,BRB - BANCO DE BRASILIA S.A.
2,00038121,SELIC,Desconhecido,BANCO CENTRAL DO BRASIL - SELIC
3,00038166,BACEN,Desconhecido,BANCO CENTRAL DO BRASIL
4,00122327,SANTINVEST S.A. - CFI,539.0,"SANTINVEST S.A. - CREDITO, FINANCIAMENTO E INV..."


Tabela 2: Bancos com Código Ímpar

In [31]:
# Convertendo 'Codigo_Banco' para numérico
df_principal['Codigo_Banco'] = pd.to_numeric(df_principal['Codigo_Banco'], errors='coerce')

# Remover linhas onde 'Codigo_Banco' seja nulo
df_principal = df_principal.dropna(subset=['Codigo_Banco'])

# Garantir que 'Codigo_Banco' seja inteiro
df_principal['Codigo_Banco'] = df_principal['Codigo_Banco'].astype(int)

# Filtrar bancos com código ímpar
df_tabela2 = df_principal[df_principal['Codigo_Banco'] % 2 != 0]
print("Tabela 2: Bancos com Código Ímpar")
display(df_tabela2.head())

Tabela 2: Bancos com Código Ímpar


,ISPB,Nome_Banco,Codigo_Banco,Nome_Completo
0,00000000,BCO DO BRASIL S.A.,1,BANCO DO BRASIL S.A.
4,00122327,SANTINVEST S.A. - CFI,539,"SANTINVEST S.A. - CREDITO, FINANCIAMENTO E INV..."
8,00329598,ÍNDIGO INVESTIMENTOS DTVM LTDA.,407,ÍNDIGO INVESTIMENTOS DISTRIBUIDORA DE TÍTULOS ...
11,00416968,BANCO INTER,77,BANCO INTER S.A.
12,00460065,COLUNA S.A. DTVM,423,COLUNA S/A DISTRIBUIDORA DE TITULOS E VALORES ...


Tabela 3: Bancos com Nome Reduzido

In [33]:
# Filtrar bancos com nomes menores que 10 caracteres
df_tabela3 = df_principal[df_principal['Nome_Banco'].str.len() < 10]
print("Tabela 3: Bancos com Nome Reduzido")
display(df_tabela3.head())

Tabela 3: Bancos com Nome Reduzido


,ISPB,Nome_Banco,Codigo_Banco,Nome_Completo
5,00204963,CCR SEARA,430,COOPERATIVA DE CREDITO RURAL SEARA - CREDISEARA
52,03973814,SERVICOOP,190,SERVICOOP - COOPERATIVA DE CRÉDITO DOS SERVIDO...
147,16695922,ID CTVM,439,ID CORRETORA DE TÍTULOS E VALORES MOBILIÁRIOS ...
170,22896431,PICPAY,380,PICPAY INSTITUIÇÃO DE PAGAMENTO S.A.
177,27302181,CRED-UFES,427,COOPERATIVA DE CREDITO DOS SERVIDORES DA UNIVE...


Conectar ao banco SQLite e salvar as tabelas

In [34]:
# Conectar ao banco SQLite
conexao = sqlite3.connect("bancos_brasilapi.db")

# Salvar as tabelas no banco de dados
df_tabela1.to_sql("todos_bancos", conexao, if_exists="replace", index=False)
df_tabela2.to_sql("bancos_codigo_impar", conexao, if_exists="replace", index=False)
df_tabela3.to_sql("bancos_nome_reduzido", conexao, if_exists="replace", index=False)

# Fechar conexão
conexao.close()
print("Tabelas armazenadas com sucesso no banco SQLite 'bancos_brasilapi.db'.")

Tabelas armazenadas com sucesso no banco SQLite 'bancos_brasilapi.db'.


Verificar bancos com Codigo_Banco igual a zero

In [35]:
# Verificar bancos com Codigo_Banco igual a 0
bancos_codigo_zero = df_tabela1[df_tabela1['Codigo_Banco'] == 0]

if not bancos_codigo_zero.empty:
    print("⚠️ Alerta: Existem bancos com 'Codigo_Banco' igual a 0:")
    print(bancos_codigo_zero)
else:
    print("Nenhum banco com 'Codigo_Banco' igual a 0 encontrado.")

Nenhum banco com 'Codigo_Banco' igual a 0 encontrado.
